In [1]:
import os
import pandas as pd
import numpy as np
from Bio import SeqIO

# Part 1: Analysis

In [2]:
# Inputting data
alpha_seqs = []
for record in SeqIO.parse("assignment1-alpha_seq-1.fasta", "fasta"):
    alpha_seqs.append(str(record.seq))
    
beta_seqs = []
for record in SeqIO.parse("assignment1-beta_seq-1.fasta", "fasta"):
    beta_seqs.append(str(record.seq))
    
# Generates lists for residues (20 elements) and residue pairs (400 elements).
residuepairs = []
residues = ["A", "C", "D", "E", "F",
        "G", "H", "I", "K", "L",
        "M", "N", "P", "Q", "R",
        "S", "T", "V", "W", "Y"]

for i in residues:
    for j in residues:
        residuepairs.append(i + j)
        
# Removes "U" from sequences
def cleaner(seq):
    ans = ""
    for i in seq:
        if i != "U":
            ans += i
    return ans

alpha_seqs = [cleaner(seq) for seq in alpha_seqs]
beta_seqs = [cleaner(seq) for seq in beta_seqs]

In [3]:
# Takes in a sequence and calculates the amino acid composition
def aacomp(seq):
    aa = {
        "A": 0, "C": 0, "D": 0, "E": 0, "F": 0,
        "G": 0, "H": 0, "I": 0, "K": 0, "L": 0,
        "M": 0, "N": 0, "P": 0, "Q": 0, "R": 0,
        "S": 0, "T": 0, "V": 0, "W": 0, "Y": 0,
        "U": 0
    }
    sum = len(seq) - aa["U"]
    for i in seq:
        aa[i] += 1
    temp = [aa[j] / sum for j in aa]
    return temp[:-1]

alpha_comps = [aacomp(i) for i in alpha_seqs]
beta_comps = [aacomp(i) for i in beta_seqs]

alpha_comps_df = pd.DataFrame(alpha_comps, columns = residues)
beta_comps_df = pd.DataFrame(beta_comps, columns = residues)

alpha_means = alpha_comps_df.mean()
alpha_var = alpha_comps_df.var()

beta_means = beta_comps_df.mean()
beta_var = beta_comps_df.var()

fdr = ((alpha_means - beta_means) ** 2) / (alpha_var + beta_var)

# Q2, Q3
# print("Alpha means are:")
# print(alpha_means)
# print("Beta means are:")
# print(beta_means)
# print("Fisher Discriminant Ratios are:")
# print(fdr)
# print(beta_var)

In [4]:
def residue_pair_prefs(seq):
    num = len(seq) - 1
    freq = {}
    for index, element in enumerate(residuepairs):
        freq[element] = 0
    for i in range(num):
        freq[seq[i:i+2]] += 1
    ans = [freq[i] / num for i in freq]
    return ans

alpha_residue_pairs = [residue_pair_prefs(i) for i in alpha_seqs]
beta_residue_pairs = [residue_pair_prefs(i) for i in beta_seqs]

alpha_residues_df = pd.DataFrame(
    alpha_residue_pairs,
    columns=residuepairs)

beta_residues_df = pd.DataFrame(
    beta_residue_pairs,
    columns=residuepairs)

alpha_residue_means = alpha_residues_df.mean()
alpha_residue_var = alpha_residues_df.var()

beta_residue_means = beta_residues_df.mean()
beta_residue_var = beta_residues_df.var()

residue_fdr = ((alpha_residue_means - beta_residue_means) ** 2) / (alpha_residue_var + beta_residue_var)

top_alpha_pairs = alpha_residue_means.nlargest(5)
top_beta_pairs = beta_residue_means.nlargest(5)
top_residue_pairs = residue_fdr.nlargest(5)

# Q4
# print("Top Alpha pairs are:")
# print(top_alpha_pairs)
# print("Top Beta pairs are:")
# print(top_beta_pairs)
# print("Top Fisher Discriminant Ratios are:")
# print(top_residue_pairs)

# Part 2: Discrimination

In [5]:
# We will use alpha_comps, beta_comps, alpha_means, beta_means from above
def predictor(seq_comp):
    # Will predict as Alpha
    sigma_tmh = 0
    sigma_tmb = 0
    for i in range(20):
        sigma_tmh += abs(seq_comp[i] - alpha_means[i])
        sigma_tmb += abs(seq_comp[i] - beta_means[i])
    if sigma_tmh < sigma_tmb:
        return 1
    else:
        return 0
    
true_positives = 0
for i in alpha_comps:
    true_positives += predictor(i)
false_negatives = len(alpha_comps) - true_positives

false_positives = 0
for i in beta_comps:
    false_positives += predictor(i)
true_negatives = len(beta_comps) - false_positives

# Q5, Q6, Q7
# print(true_positives, false_positives, true_negatives, false_negatives)

In [6]:
alpha_comps_half = alpha_comps[:len(alpha_comps) // 2]
beta_comps_half = beta_comps[:len(beta_comps) // 2]

alpha_comps_half_df = pd.DataFrame(alpha_comps_half, columns = residues)
beta_comps_half_df = pd.DataFrame(beta_comps_half, columns = residues)

alpha_half_mean = alpha_comps_half_df.mean()
beta_half_mean = beta_comps_half_df.mean()

def predictor_half(seq_comp):
    # Will predict as Alpha
    sigma_tmh = 0
    sigma_tmb = 0
    for i in range(20):
        sigma_tmh += abs(seq_comp[i] - alpha_half_mean[i])
        sigma_tmb += abs(seq_comp[i] - beta_half_mean[i])
    if sigma_tmh < sigma_tmb:
        return 1
    else:
        return 0
    
true_positives_half = 0
for i in alpha_comps_half:
    true_positives_half += predictor(i)
false_negatives_half = len(alpha_comps_half) - true_positives_half

false_positives_half = 0
for i in beta_comps_half:
    false_positives_half += predictor(i)
true_negatives_half = len(beta_comps_half) - false_positives_half

# Q9
# print(true_positives_half, false_positives_half, true_negatives_half, false_negatives_half)

298 8 64 52


# Part 3: Comparison of different features

In [7]:
# We will use alpha_residue_pairs, beta_residue_pairs, alpha_residue_means, beta_residue_means from above
def predictor_respairs(seq_comp):
    # Will predict as Alpha
    sigma_tmh = 0
    sigma_tmb = 0
    for i in range(400):
        sigma_tmh += abs(seq_comp[i] - alpha_residue_means[i])
        sigma_tmb += abs(seq_comp[i] - beta_residue_means[i])
    if sigma_tmh < sigma_tmb:
        return 1
    else:
        return 0
    
true_positives_respairs = 0
for i in alpha_residue_pairs:
    true_positives_respairs += predictor_respairs(i)
false_negatives_respairs = len(alpha_comps) - true_positives_respairs

false_positives_respairs = 0
for i in beta_residue_pairs:
    false_positives_respairs += predictor_respairs(i)
true_negatives_respairs = len(beta_comps) - false_positives_respairs

# Q(a)
# print(true_positives_respairs, false_positives_respairs, true_negatives_respairs, false_negatives_respairs)

630 9 135 70


In [8]:
alpha_respairs_half = alpha_residue_pairs[:len(alpha_comps) // 2]
beta_respairs_half = beta_residue_pairs[:len(beta_comps) // 2]

alpha_respairs_half_df = pd.DataFrame(alpha_respairs_half, columns = residuepairs)
beta_respairs_half_df = pd.DataFrame(beta_respairs_half, columns = residuepairs)

alpha_respairs_half_mean = alpha_respairs_half_df.mean()
beta_respairs_half_mean = beta_respairs_half_df.mean()

def predictor_respairs_half(seq_comp):
    # Will predict as Alpha
    sigma_tmh = 0
    sigma_tmb = 0
    for i in range(400):
        sigma_tmh += abs(seq_comp[i] - alpha_respairs_half_mean[i])
        sigma_tmb += abs(seq_comp[i] - beta_respairs_half_mean[i])
    if sigma_tmh < sigma_tmb:
        return 1
    else:
        return 0
    
true_positives_respairs_half = 0
for i in alpha_respairs_half:
    true_positives_respairs_half += predictor_respairs_half(i)
false_negatives_respairs_half = len(alpha_comps_half) - true_positives_respairs_half

false_positives_respairs_half = 0
for i in beta_respairs_half:
    false_positives_respairs_half += predictor_respairs_half(i)
true_negatives_respairs_half = len(beta_comps_half) - false_positives_respairs_half

# Q(a)
# print(true_positives_respairs_half, false_positives_respairs_half, true_negatives_respairs_half, false_negatives_respairs_half)

317 6 66 33


In [11]:
# Classifier with all amino acid and residue pair features (total 420 features).
def all_predictor(seq_comp, seq_pairs):
    sigma_tmh = 0
    sigma_tmb = 0
    for i in range(20):
        sigma_tmh += abs(seq_comp[i] - alpha_means[i])
        sigma_tmb += abs(seq_comp[i] - beta_means[i])
    for i in range(400):
        sigma_tmh += abs(seq_pairs[i] - alpha_residue_means[i])
        sigma_tmb += abs(seq_pairs[i] - beta_residue_means[i])
    if sigma_tmh < sigma_tmb:
        return 1
    else:
        return 0
    
all_true_positives = 0
for i in range(700):
    all_true_positives += all_predictor(alpha_comps[i], alpha_residue_pairs[i])
all_false_negatives = len(alpha_comps) - all_true_positives

all_false_positives = 0
for i in range(144):
    all_false_positives += all_predictor(beta_comps[i], beta_residue_pairs[i])
all_true_negatives = len(beta_comps) - all_false_positives

# Q(b)
# print(all_true_positives, all_false_positives, all_true_negatives, all_false_negatives)

617 11 133 83


In [21]:
# Classifier with only selected features (total 7 features).
def select_predictor(seq_comp, seq_pairs):
    sigma_tmh = 0
    sigma_tmb = 0
    for i in [2, 11]: # index for 2 most important amino acid
        sigma_tmh += abs(seq_comp[i] - alpha_means[i])
        sigma_tmb += abs(seq_comp[i] - beta_means[i])
    for i in [189, 187, 149, 51, 349]: # indexes for top 5 most important pairs
        sigma_tmh += abs(seq_pairs[i] - alpha_residue_means[i])
        sigma_tmb += abs(seq_pairs[i] - beta_residue_means[i])
    if sigma_tmh < sigma_tmb:
        return 1
    else:
        return 0
    
select_true_positives = 0
for i in range(700):
    select_true_positives += select_predictor(alpha_comps[i], alpha_residue_pairs[i])
select_false_negatives = len(alpha_comps) - select_true_positives

select_false_positives = 0
for i in range(144):
    select_false_positives += select_predictor(beta_comps[i], beta_residue_pairs[i])
select_true_negatives = len(beta_comps) - select_false_positives

# Q(c)
# print(select_true_positives, select_false_positives, select_true_negatives, select_false_negatives)

601 15 129 99
